In [46]:
import h2o
import src.dataCleaning as dc
from h2o.automl import H2OAutoML

In [47]:
diamonds =  pd.read_csv('./diamonds-datamad1019/data.csv/data.csv')

In [48]:
diamonds = dc.dfCleaning(diamonds)

In [49]:
diamonds.head()

,carat,depth,table,price,color_num,cut_num,clarity_num,volume
0,2.26,61.9,57.0,12831,4.0,5.0,2.0,366.903680
1,2.43,63.2,57.0,16170,3.0,3.0,2.0,392.176400
2,0.80,61.0,57.0,2797,5.0,4.0,2.0,133.001901
3,0.40,63.3,60.0,630,5.0,5.0,1.0,64.059840
4,0.31,61.6,55.0,698,4.0,5.0,4.0,51.797610


In [50]:
h2o.init(nthreads = -1, max_mem_size = 26)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,21 mins 06 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_mariobru_khs42j
H2O cluster total nodes:,1
H2O cluster free memory:,24.19 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [51]:
diamonds_h2o=h2o.H2OFrame(diamonds)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [52]:
diamonds_h2o.head()

carat,depth,table,price,color_num,cut_num,clarity_num,volume
2.26,61.9,57,12831,4,5,2,366.904
2.43,63.2,57,16170,3,3,2,392.176
0.8,61,57,2797,5,4,2,133.002
0.4,63.3,60,630,5,5,1,64.0598
0.31,61.6,55,698,4,5,4,51.7976
0.53,60.5,60,1132,5,3,2,85.5027
0.7,60.8,61,1987,7,4,2,110.716
1.13,64,58,3952,5,2,2,179.758
0.35,60.9,55,614,6,5,3,57.3448
1.01,61.4,66,5062,3,1,5,153.103


In [55]:
y_columns = "price"
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [56]:
train, test=diamonds_h2o.split_frame(ratios = [.95])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [57]:
aml_ti = H2OAutoML(max_runtime_secs= 360,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [58]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_2_AutoML_20191210_002405,288044,536.697,288044,283.494,0.112287
GBM_3_AutoML_20191210_002405,293044,541.335,293044,286.631,0.11575
GBM_1_AutoML_20191210_002405,293150,541.434,293150,288.829,0.121441
XGBoost_3_AutoML_20191210_002405,295341,543.453,295341,289.085,0.123173
XGBoost_1_AutoML_20191210_002405,300015,547.736,300015,285.257,0.1112
GBM_5_AutoML_20191210_002405,300587,548.258,300587,287.855,0.113531
XGBoost_2_AutoML_20191210_002405,306727,553.829,306727,301.598,0.136265
GBM_4_AutoML_20191210_002405,319151,564.935,319151,307.908,0.146205
XGBoost_grid_1_AutoML_20191210_002405_model_1,347486,589.48,347486,298.181,0.111643
DeepLearning_1_AutoML_20191210_002405,374817,612.223,374817,343.128,nan


In [59]:
pred_automl = aml_ti.leader.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [60]:
pred_automl

predict
6168.08
3253.45
817.446
4317.02
2245.88
6115.4
12600.2
1902.6
4603.95
723.506


In [61]:
y_test

price
6670
2732
727
4200
2159
6146
11640
1819
5127
844


In [62]:
#Probamos con test

In [63]:
diamonds_test =  pd.read_csv('./diamonds-datamad1019/test.csv')

In [64]:
diamonds_test = dc.dfCleaning(diamonds_test)

In [65]:
diamonds_h2o=h2o.H2OFrame(diamonds_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [66]:
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [67]:
X_train=diamonds_h2o[x_columns]

In [68]:
pred_automl = aml_ti.leader.predict(X_train)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [69]:
pred_automl

predict
1293.74
6711.9
1630.83
4024.92
1800.82
10130.7
812.562
999.084
699.92
14165


In [70]:
submission = pred_automl.as_data_frame()

In [71]:
submission

,predict
0,1293.736308
1,6711.895912
2,1630.830455
3,4024.922385
4,1800.819345
...,...
13480,2209.954432
13481,2038.879554
13482,11142.433160
13483,465.113958


In [72]:
submission.rename(columns={"predict": "price"}, inplace=True)
submission

,price
0,1293.736308
1,6711.895912
2,1630.830455
3,4024.922385
4,1800.819345
...,...
13480,2209.954432
13481,2038.879554
13482,11142.433160
13483,465.113958


In [73]:
submission.price = submission.price.astype(int)


In [74]:
submission

,price
0,1293
1,6711
2,1630
3,4024
4,1800
...,...
13480,2209
13481,2038
13482,11142
13483,465


In [75]:
submission.to_csv('./diamonds-datamad1019/submission.csv')